In [1]:
import pandas as pd
import numpy as np
import requests
from mlserver.codecs import PandasCodec

red = pd.read_csv(
    "https://storage.googleapis.com/seldon-datasets/wine_quality/winequality-red.csv", sep=';'
)
white = pd.read_csv(
    "https://storage.googleapis.com/seldon-datasets/wine_quality/winequality-white.csv", sep=';'
)
white = white.drop(["quality"], axis=1)
white = white[:500].astype(np.float64)



In [2]:

# params = {
#     "detector_name": "WineDriftDetector",
#     "data_type": "tabular",
# }
# training_request = PandasCodec.encode_request(white)

# endpoint = "http://localhost:8080/v2/models/fit/"
# response = requests.post(endpoint, json=training_request.dict(), params=params)

In [3]:
import pandas as pd
import numpy as np
import requests
from mlserver.codecs import PandasCodec
import tensorflow

red = pd.read_csv(
    "https://storage.googleapis.com/seldon-datasets/wine_quality/winequality-red.csv",
    sep=";",
)
white = pd.read_csv(
    "https://storage.googleapis.com/seldon-datasets/wine_quality/winequality-white.csv",
    sep=";",
)
white = white.drop(["quality"], axis=1)
white = white[:500].astype(np.float64)


# params = {
#     "detector_name": "WineDriftDetector",
#     "data_type": "tabular",
# }
# training_request = PandasCodec.encode_request(white)

# endpoint = "http://localhost:8080/v2/models/fit/"
# response = requests.post(endpoint, json=training_request.dict(), params=params)

import torch


from alibi_detect.cd import MMDDriftOnline
from sklearn.decomposition import PCA

torch.manual_seed(0)
np.random.seed(0)

pca = PCA(2)
pca.fit(white)

ert = 50
window_size = 10

detector = MMDDriftOnline(
    white,
    ert,
    window_size,
    backend="tensorflow",
    preprocess_fn=pca.transform,
    n_bootstraps=2500,
)


/opt/homebrew/Caskroom/miniforge/base/envs/drifting/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: 

In [18]:
white.shape

(5, 11)

In [7]:
columns = white.columns.values.tolist()[:-1]

In [8]:
n_white, n_red = white.shape[0], red.shape[0]

col_maxes = white.max(axis=0)
white, red = white / col_maxes, red / col_maxes

white = white.iloc[np.random.permutation(n_white)]
red = red.iloc[np.random.permutation(n_red)]

X = white[columns]
X_corr = red[columns]

X_train = X[:(n_white//2)]
X_ref = X[(n_white//2):(3*n_white//4)]
X_h0 = X[(3*n_white//4):]

In [9]:
pca = PCA(2)
pca.fit(X_train)

In [ ]:
enc_h0 = pca.transform(X_h0)
enc_h1 = pca.transform(X_corr)

plt.scatter(enc_h0[:,0], enc_h0[:,1], alpha=0.2, color='green', label='white wine')
plt.scatter(enc_h1[:,0], enc_h1[:,1], alpha=0.2, color='red', label='red wine')
plt.legend(loc='upper right')
plt.show()

In [ ]:
import json

X_train = X_train[:10].astype(np.float64)
inference_request = PandasCodec.encode_request(X_train)
json.dumps(inference_request)

In [72]:
# import requests

# inference_request = {
#     "inputs": [
#         {
#           "name": column,
#           "shape": [10],
#           "datatype": "FP32",
#           "data": X_train[:10, i].tolist(),
#         }
#         for i, column in enumerate(columns)
        
#     ]
# }

# params = {
#     "detector_name": "WineDetector",
#     "data_type": "tabular",
# }

# endpoint = "http://localhost:8080/v2/models/fit/"
# response = requests.post(endpoint, json=inference_request, params=params)

# response.json()

In [77]:
inference_request

{'inputs': [{'name': 'fixed acidity',
   'shape': [10],
   'datatype': 'FP32',
   'data': [0.51408451795578,
    0.49295774102211,
    0.5352112650871277,
    0.35211268067359924,
    0.4225352108478546,
    0.6901408433914185,
    0.5845070481300354,
    0.5422534942626953,
    0.5070422291755676,
    0.43661969900131226]},
  {'name': 'volatile acidity',
   'shape': [10],
   'datatype': 'FP32',
   'data': [0.290909081697464,
    0.2818181812763214,
    0.12727272510528564,
    0.26363635063171387,
    0.2545454502105713,
    0.8454545140266418,
    0.1818181872367859,
    0.2818181812763214,
    0.16363637149333954,
    0.22727271914482117]},
  {'name': 'citric acid',
   'shape': [10],
   'datatype': 'FP32',
   'data': [0.21084336936473846,
    0.15662650763988495,
    0.4457831382751465,
    0.32530122995376587,
    0.1325301229953766,
    0.27108433842658997,
    0.4457831382751465,
    0.21686747670173645,
    0.1867469996213913,
    0.28313252329826355]},
  {'name': 'residual suga

In [76]:
# inference_request

In [ ]:
def time_run(cd, X, window_size):
    n = X.shape[0]
    perm = np.random.permutation(n)
    t = 0
    cd.reset()
    while True:
        pred = cd.predict(X[perm[t%n]])
        if pred['data']['is_drift'] == 1:
            return t
        else:
            t += 1
